# German Credit Analysis, Using DNN

Some necessary libraries:

In [12]:
from __future__ import division
from math import log
import pandas as pd

### Read in the data

In [13]:
german = pd.read_csv('MGMT635_GermanCreditData.csv')

In [14]:
german.head()

,status,duration,history,purpose,amount,savings,employment,rate,personal,guarantor,...,propert,age,other_debt,housing_type,existing_credit_bank,job,dependents,phone,foreign,target
0,11,6,34,43,1169,65,75,4,93,101,...,121,67,143,152,2,173,1,192,201,1
1,12,48,32,43,5951,61,73,2,92,101,...,121,22,143,152,1,173,1,191,201,2
2,14,12,34,46,2096,61,74,2,93,101,...,121,49,143,152,1,172,2,191,201,1
3,11,42,32,42,7882,61,74,2,93,103,...,122,45,143,153,1,173,2,191,201,1
4,11,24,33,40,4870,61,73,3,93,101,...,124,53,143,153,2,173,2,191,201,2


In [15]:
attribute_dict = {1:['checking', 'history', 'purpose', 'savings',
                    'employment', 'status', 'guarantor','propert', 'other_debt', 'housing_type', 
                    'job', 'phone', 'foreign'], 
                  2:['duration', 'amount', 'rate', 'residence_time','age', 'existing_credit_bank',
                     'dependents',]}

In [16]:
german.describe()

,status,duration,history,purpose,amount,savings,employment,rate,personal,guarantor,...,propert,age,other_debt,housing_type,existing_credit_bank,job,dependents,phone,foreign,target
count,1000.000000,1000.000000,1000.00000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.00000,1000.000000,...,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000
mean,12.577000,20.903000,32.54500,47.148000,3271.258000,62.105000,73.384000,2.973000,92.68200,101.145000,...,122.358000,35.546000,142.675000,151.929000,1.407000,172.904000,1.155000,191.404000,201.037000,1.300000
std,1.257638,12.058814,1.08312,40.095333,2822.736876,1.580023,1.208306,1.118715,0.70808,0.477706,...,1.050209,11.375469,0.705601,0.531264,0.577654,0.653614,0.362086,0.490943,0.188856,0.458487
min,11.000000,4.000000,30.00000,40.000000,250.000000,61.000000,71.000000,1.000000,91.00000,101.000000,...,121.000000,19.000000,141.000000,151.000000,1.000000,171.000000,1.000000,191.000000,201.000000,1.000000
25%,11.000000,12.000000,32.00000,41.000000,1365.500000,61.000000,73.000000,2.000000,92.00000,101.000000,...,121.000000,27.000000,143.000000,152.000000,1.000000,173.000000,1.000000,191.000000,201.000000,1.000000
50%,12.000000,18.000000,32.00000,42.000000,2319.500000,61.000000,73.000000,3.000000,93.00000,101.000000,...,122.000000,33.000000,143.000000,152.000000,1.000000,173.000000,1.000000,191.000000,201.000000,1.000000
75%,14.000000,24.000000,34.00000,43.000000,3972.250000,63.000000,75.000000,4.000000,93.00000,101.000000,...,123.000000,42.000000,143.000000,152.000000,2.000000,173.000000,1.000000,192.000000,201.000000,2.000000
max,14.000000,72.000000,34.00000,410.000000,18424.000000,65.000000,75.000000,4.000000,94.00000,103.000000,...,124.000000,75.000000,143.000000,153.000000,4.000000,174.000000,2.000000,192.000000,202.000000,2.000000


# Feature Selection
Using information theory, as outlined in Provost and Fawcett (2013). We calculated the information gained for each of the attributes. As an example we will display the calculations for the attribute `status`.

First, based on the data set, the overall probabilities of good credit or bad credit can be calculated:

In [17]:
p_parent_good = german.where(german.target==1).dropna().shape[0]/german.shape[0]
print("Probability of good credit = {}".format(p_parent_good))
p_parent_bad = german.where(german.target==2).dropna().shape[0]/german.shape[0]
print("Probability of bad credit = {}".format(p_parent_bad))

Probability of good credit = 0.7
Probability of bad credit = 0.3


The total entropy of the data set is calculated:

In [18]:
parent_entropy = - (p_parent_good * log(p_parent_good, 2) + p_parent_bad * log(p_parent_bad, 2))
print("Parent Entropy = {}".format(parent_entropy))

Parent Entropy = 0.8812908992306927


In order to understand how informative the attribute is we need to calculate the information gained. This is done by calculating how much the attribute reduces the entropy of the segmentations created by splitting the data set along the values of the attribute.

In [19]:
feature_values_for_status = [11,12,13,14]

status_value_series = {}
#Split the data set along the values of the attributes.
for value in feature_values_for_status:
    status_value_series[value] = german.where(german['status']==value).dropna().target

IG_children = 0
for key, series in status_value_series.items():
    p_status_value = series.shape[0] / german.shape[0]
    p_series_good = series.where(series==1).dropna().shape[0] / series.shape[0]
    p_series_bad = series.where(series==2).dropna().shape[0] / series.shape[0]
    entropy_child = -(p_series_good * log(p_series_good, 2) + p_series_bad * log(p_series_bad, 2))
    IG_children = IG_children + (p_status_value * entropy_child)
    print("Probability of value {}: {}".format(key, p_status_value))
    print("Probability for value {}, to have good credit: {}".format(key, p_series_good))
    print("Probability for value {}, to have bad credit: {}".format(key, p_series_bad))
    print("Entropy of child with value {}: {}".format(key, entropy_child))
    print("--------------------------------------------------------")

Probability of value 11: 0.274
Probability for value 11, to have good credit: 0.5072992700729927
Probability for value 11, to have bad credit: 0.4927007299270073
Entropy of child with value 11: 0.9998462628494693
--------------------------------------------------------
Probability of value 12: 0.269
Probability for value 12, to have good credit: 0.6096654275092936
Probability for value 12, to have bad credit: 0.3903345724907063
Entropy of child with value 12: 0.9650151205034324
--------------------------------------------------------
Probability of value 13: 0.063
Probability for value 13, to have good credit: 0.7777777777777778
Probability for value 13, to have bad credit: 0.2222222222222222
Entropy of child with value 13: 0.7642045065086203
--------------------------------------------------------
Probability of value 14: 0.394
Probability for value 14, to have good credit: 0.883248730964467
Probability for value 14, to have bad credit: 0.116751269035533
Entropy of child with value 14

We use the sum of the products of the children entropies and probabilities and subtract it from the entropy of the parent:

In [20]:
IG = parent_entropy - IG_children
print("Information Gain for attribute status: {}".format(IG))

Information Gain for attribute status: 0.09473884155263945


##### Noted as most relevant attributes 

In [21]:
german2 = german[['duration', 'history','amount', 'savings',
       'status','housing_type', 'foreign','target']]

In [22]:
german2.head()


,duration,history,amount,savings,status,housing_type,foreign,target
0,6,34,1169,65,11,152,201,1
1,48,32,5951,61,12,152,201,2
2,12,34,2096,61,14,152,201,1
3,42,32,7882,61,11,153,201,1
4,24,33,4870,61,11,153,201,2


### Chop up the data (training, testing)

In [23]:
from sklearn.model_selection import train_test_split

In [24]:
x_data = german2.drop('target',axis=1)

In [25]:
y_labels = german2['target']

In [26]:
X_train, X_test, y_train, y_test = train_test_split(x_data, y_labels, test_size=0.025, random_state=101)

In [27]:
german.columns

Index(['status', 'duration', 'history', 'purpose', 'amount', 'savings',
       'employment', 'rate', 'personal', 'guarantor', 'residence_time',
       'propert', 'age', 'other_debt', 'housing_type', 'existing_credit_bank',
       'job', 'dependents', 'phone', 'foreign', 'target'],
      dtype='object')

In [28]:
import tensorflow as tf

### Assign feature columns for TensorFlow (numeric, categorical, etc.)

In [29]:
history = tf.feature_column.numeric_column("history")
savings = tf.feature_column.numeric_column("savings")
status = tf.feature_column.numeric_column("status")
housing_type = tf.feature_column.numeric_column("housing_type")
foreign = tf.feature_column.numeric_column("foreign")
duration = tf.feature_column.numeric_column("duration")
amount = tf.feature_column.numeric_column("amount")

In [30]:
for attribute in attribute_dict.values():
    print(attribute)

['checking', 'history', 'purpose', 'savings', 'employment', 'status', 'guarantor', 'propert', 'other_debt', 'housing_type', 'job', 'phone', 'foreign']
['duration', 'amount', 'rate', 'residence_time', 'age', 'existing_credit_bank', 'dependents']


In [31]:
feat_cols = [duration, history,amount, savings,
            status,housing_type, foreign]

### Create model and Input the data

In [32]:
input_func = tf.estimator.inputs.pandas_input_fn(x=X_train,y=y_train,batch_size=10,num_epochs=1000,shuffle=True)

In [33]:
dnn_model = tf.estimator.DNNClassifier(hidden_units=[9,9,9],feature_columns=feat_cols, n_classes=4)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_summary_steps': 100, '_model_dir': 'C:\\Users\\JMORR_~1\\AppData\\Local\\Temp\\tmp6_dyn0ox', '_keep_checkpoint_every_n_hours': 10000, '_keep_checkpoint_max': 5, '_tf_random_seed': 1, '_log_step_count_steps': 100, '_save_checkpoints_steps': None, '_session_config': None, '_save_checkpoints_secs': 600}


In [34]:
hist = dnn_model.train(input_fn=input_func,steps=1000)

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into C:\Users\JMORR_~1\AppData\Local\Temp\tmp6_dyn0ox\model.ckpt.
INFO:tensorflow:step = 1, loss = 5451.99
INFO:tensorflow:global_step/sec: 167.825
INFO:tensorflow:step = 101, loss = 7.13607 (0.627 sec)
INFO:tensorflow:global_step/sec: 143.61
INFO:tensorflow:step = 201, loss = 7.09184 (0.698 sec)
INFO:tensorflow:global_step/sec: 133.357
INFO:tensorflow:step = 301, loss = 7.47571 (0.733 sec)
INFO:tensorflow:global_step/sec: 146.154
INFO:tensorflow:step = 401, loss = 8.62339 (0.684 sec)
INFO:tensorflow:global_step/sec: 133.65
INFO:tensorflow:step = 501, loss = 6.40877 (0.760 sec)
INFO:tensorflow:global_step/sec: 161.241
INFO:tensorflow:step = 601, loss = 5.13535 (0.604 sec)
INFO:tensorflow:global_step/sec: 146.154
INFO:tensorflow:step = 701, loss = 6.0388 (0.708 sec)
INFO:tensorflow:global_step/sec: 135.094
INFO:tensorflow:step = 801, loss = 5.83883 (0.736 sec)
INFO:tensorflow:global_step/sec: 138.847
IN

### Evaluate the predictions

In [35]:
eval_input_func = tf.estimator.inputs.pandas_input_fn(x=X_test,y=y_test,batch_size=15,num_epochs=1,shuffle=False)

In [36]:
evaluation = dnn_model.evaluate(eval_input_func)

INFO:tensorflow:Starting evaluation at 2019-02-27-12:02:21
INFO:tensorflow:Restoring parameters from C:\Users\JMORR_~1\AppData\Local\Temp\tmp6_dyn0ox\model.ckpt-1000
INFO:tensorflow:Finished evaluation at 2019-02-27-12:02:24
INFO:tensorflow:Saving dict for global step 1000: accuracy = 0.76, average_loss = 0.552021, global_step = 1000, loss = 6.90026


In [37]:
tf_eval = pd.DataFrame([{"Accuracy": evaluation['accuracy'],"Loss":evaluation["loss"],"Average Loss": evaluation["average_loss"]}])

In [38]:
predictions = list(dnn_model.predict(eval_input_func))

INFO:tensorflow:Restoring parameters from C:\Users\JMORR_~1\AppData\Local\Temp\tmp6_dyn0ox\model.ckpt-1000


In [39]:
final_preds = []
for pred in predictions:
    final_preds.append(pred['class_ids'][0])
    
test_targets = []
for y in y_test:
    test_targets.append(y)

In [40]:
results = pd.DataFrame({"Predicted Result": final_preds,"Actual Result":test_targets})
score = 0
index = 0

for x in results['Actual Result']:
    if x == 2 and results["Predicted Result"][index] == 1:
        score = score + 5
    if x == 1 and results["Predicted Result"][index] == 2:
        score = score + 1
    else:
        score = score + 0
    index += 1
score

30

In [41]:
y_actu = pd.Series(test_targets, name='Actual')
y_pred = pd.Series(final_preds, name='Predicted')

In [42]:
df_confusion = pd.crosstab(y_actu, y_pred, rownames=['Actual'], colnames=['Predicted'], margins=True)
df_confusion

Predicted,1,2,All
Actual,,,
1,18,0,18
2,6,1,7
All,24,1,25


In [43]:
with pd.ExcelWriter('Results.xlsx') as writer:
    results.to_excel(writer, sheet_name='Output')
    tf_eval.to_excel(writer, sheet_name='Evaluation')
    df_confusion.to_excel(writer, sheet_name='Confusion Matrix')

In [44]:
import os
file = "Results.xlsx"
os.startfile(file)